In [1]:
import sys
import re

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

sys.path.append('../src/')

from visualization.visualize import get_timeseries

In [2]:
selected = ["621000", "620000", "621040"]

data_raw = pd.read_parquet("../data/processed/pks.parquet")
timerange = [min(data_raw.year), max(data_raw.year)]
data_raw

,key,year,state,label,index,count,freq,attempts,clearance,label_change,color
0,****00,2013,Baden-Württemberg,Diebstahl insgesamt und zwar:,5599,208604,1973.700000,21196,64409,True,#bf5f00
1,****00,2013,Bayern,Diebstahl insgesamt und zwar:,5600,192429,1537.000000,16300,70111,True,#bf5f00
2,****00,2013,Berlin,Diebstahl insgesamt und zwar:,5601,226279,6704.100000,22016,48041,True,#bf5f00
3,****00,2013,Brandenburg,Diebstahl insgesamt und zwar:,5602,83537,3410.400000,9144,23639,True,#bf5f00
4,****00,2013,Bremen,Diebstahl insgesamt und zwar:,5603,39467,6027.600000,4932,9743,True,#bf5f00
...,...,...,...,...,...,...,...,...,...,...,...
164394,980100,2020,Saarland,Cybercrime im engeren Sinne,18779,982,99.504807,137,443,False,#bf0002
164395,980100,2020,Sachsen,Cybercrime im engeren Sinne,18780,2295,56.360912,216,1627,False,#bf0002
164396,980100,2020,Sachsen-Anhalt,Cybercrime im engeren Sinne,18781,5348,243.668847,875,1663,False,#bf0002
164397,980100,2020,Schleswig-Holstein,Cybercrime im engeren Sinne,18782,1625,55.961675,320,818,False,#bf0002


In [3]:
selected = [{"state": "Bund", "key": "621000"},
            {"state": "Bund", "key": "620000"},
            {"state": "Bund", "key": "100000"},
            ]

df = pd.concat([
    data_raw.loc[(data_raw.state.eq(entry["state"])) & (data_raw.key.eq(entry["key"]))] for entry in selected
])

df = df[["key", "state", "year", "label", "count", "clearance", "color"]]
df["unsolved"] = df["count"] - df["clearance"]
df = pd.melt(df,
                  id_vars=["key", "state", "year", "label", "color"],
                  value_vars=["count", "unsolved"],
                  )
colormap = {i: grp.loc[grp.index[0], "color"] for i, grp in df.groupby("key")}

df

,key,state,year,label,color,variable,value
0,621000,Bund,2013,"Widerstand gegen die Staatsgewalt §§ 111, 113,...",#001bbf,count,21618
1,621000,Bund,2014,"Widerstand gegen die Staatsgewalt §§ 111, 113,...",#001bbf,count,21937
2,621000,Bund,2015,"Widerstand gegen die Staatsgewalt §§ 111, 113,...",#001bbf,count,21945
3,621000,Bund,2016,"Widerstand gegen die Staatsgewalt §§ 111, 113,...",#001bbf,count,24362
4,621000,Bund,2017,"Widerstand gegen die Staatsgewalt §§ 111, 113,...",#001bbf,count,24419
5,621000,Bund,2018,Widerstand gegen und tätlicher Angriff auf die...,#001bbf,count,34168
6,621000,Bund,2019,Widerstand gegen und tätlicher Angriff auf die...,#001bbf,count,36959
7,621000,Bund,2020,Widerstand gegen und tätlicher Angriff auf die...,#001bbf,count,37778
8,621000,Bund,2021,Widerstand gegen und tätlicher Angriff auf die...,#001bbf,count,39239
9,621000,Bund,2022,Widerstand gegen und tätlicher Angriff auf die...,#001bbf,count,42013


In [4]:
fig = get_timeseries(df)
fig.show()